<a href="https://colab.research.google.com/github/okliviaf/PredictingPS/blob/master/Per_query_PS_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive
!pip install -U -q semopy
!pip install -U -q seaborn
!pip install -U -q pingouin
!pip install -U -q tensorflow-gpu==1.15.2
from tensorflow import set_random_seed
import pingouin as pg
import numpy as np
import seaborn as sns
import pandas as pd
from scipy.stats import f
import csv
from sklearn import svm
from statsmodels.stats.anova import AnovaRM
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files
from keras.models import Sequential
from keras.layers import Dense
from sklearn import preprocessing
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import optimizers
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
import numpy
from collections import namedtuple
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Dataset = namedtuple("Dataset", ["X", "y", "num_features", "num_classes"])

     |████████████████████████████████| 411.0MB 41kB/s 
     |████████████████████████████████| 512kB 40.8MB/s 
     |████████████████████████████████| 3.8MB 53.9MB/s 
ERROR: tensorflow 2.2.0rc3 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc3 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0rc3 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [0]:
downloaded = drive.CreateFile({"id": "1JSEWIrZlNa62D0JQUar6XBDO5z7yh2tO"})
downloaded.GetContentFile("FA_Binary.csv")
FA_Binary = pd.read_csv("FA_Binary.csv")

downloaded = drive.CreateFile({"id": "1XfCcEeHHWoKh0W_JXOxmdjpoPto2uZQy"})
downloaded.GetContentFile("NC_Binary.csv")
NC_Binary = pd.read_csv("NC_Binary.csv")

downloaded = drive.CreateFile({"id": "1e48W6IPytV7Nj46O_CJ3RqaRVOX4ymJU"})
downloaded.GetContentFile("Overall_Trio.csv")
Overall_Trio = pd.read_csv("Overall_Trio.csv")

In [0]:
len(FA_Binary)

575

In [0]:
attributes = [
  'no_of_actions',
  'time_query',
  'time_on_serp',
  'time_on_documents',
   'time_session_overall',
    'serp_page_viewed_to',
    'document_click_count',
    'document_click_depth',
    'document_hover_count_raw',
    'document_hover_count',
    'document_hover_depth',
    'ad_hover_count',
    'ad_hover_count_top',
    'ad_hover_count_side',
    'ad_hover_count_bot',
    'ad_click_count',
    'depth',
    'time_per_snippet',
    'time_per_document',
    'query_length',
    'query_tokens_count'
]

seed = 7
num_splits = 5
max_num_attributes = 5
feature_selector = SelectKBest(mutual_info_classif, k=max_num_attributes)

feature_scores = []

# fix random seed for reproducibility
numpy.random.seed(seed)
set_random_seed(seed)

In [0]:
# Data preparation
def dataset_preparation(dataset_df, num_classes, feature_names, apply_feature_selection=False):
    label_encoder = preprocessing.LabelEncoder()
    
    # split into input (X) and output (Y) variables
    X = preprocessing.scale(dataset_df[feature_names].to_numpy())
    y = label_encoder.fit_transform(dataset_df.iloc[:, -1].to_numpy().tolist())

    if apply_feature_selection:
        X = feature_selector.fit_transform(X, y)
        num_features = X.shape[-1]
        feature_scores.append(feature_selector.scores_)
    else:
        num_features = len(feature_names)

    return Dataset(X=X, y=y, num_features=num_features, num_classes=num_classes)

In [0]:
datasets = {
    "FA_Binary": dataset_preparation(FA_Binary, 2, attributes, apply_feature_selection=True),
    "NC_Binary": dataset_preparation(NC_Binary, 2, attributes, apply_feature_selection=True),
    "Overall_Trio": dataset_preparation(Overall_Trio, 3, attributes, apply_feature_selection=True)
}

for i, (dataset_name, dataset) in enumerate(datasets.items()):
    print("Dataset name: {}".format(dataset_name))
    indices = np.argsort(feature_scores[i])[::-1][:10]
    for ind in indices:
        print(f"- {attributes[ind]} ({feature_scores[i][ind]})")


Dataset name: FA_Binary
- no_of_actions (0.49927063236047586)
- document_hover_depth (0.04289568609635075)
- depth (0.040732476046951405)
- ad_hover_count_side (0.03037492538505293)
- document_click_count (0.026388897100060982)
- document_hover_count (0.02389865063155372)
- time_per_document (0.01495280161510304)
- ad_hover_count (0.013664022761106676)
- time_query (0.009775827409995719)
- query_tokens_count (0.006931735014752416)
Dataset name: NC_Binary
- no_of_actions (0.5076915411658265)
- ad_hover_count_side (0.037889461936959634)
- time_per_document (0.0328501411748634)
- time_per_snippet (0.023341917514465793)
- ad_hover_count_top (0.02226291850835671)
- document_hover_depth (0.021015808840531225)
- document_click_count (0.018008279427630836)
- query_length (0.016176541827350954)
- document_click_depth (0.013715992714627978)
- serp_page_viewed_to (0.013491000315722479)
Dataset name: Overall_Trio
- no_of_actions (0.698567101228561)
- document_click_count (0.0338203004430897)
- ad_

In [0]:
def nn_classification_model(input_size, output_size, hidden_sizes=[64], activation="relu"):
    """
        Creates a Feed-forward Neural Network (1-layer) with an output layer of 3 neurons.
        Every output neuron represents a specific class (low, medium, high).

        The final Softmax activation function is used to obtain a probability distribution 
        over the classes (low, medium, high)
    """
    def model_definition():
        model = Sequential()
        model.add(Dense(hidden_sizes[0], input_dim=input_size, kernel_initializer='glorot_uniform', activation=activation))

        for hidden_size in hidden_sizes[1:]:
            model.add(Dense(hidden_size, kernel_initializer='glorot_uniform', activation=activation))
            model.add(Dropout(0.35, seed=seed))

        # output layer
        model.add(Dense(output_size, kernel_initializer='glorot_uniform', activation="softmax"))
        adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)

        # Compile model
        #model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
        model.compile(loss='sparse_categorical_crossentropy',  metrics=["accuracy"], optimizer = adam)
            
        return model
    
    return model_definition

def create_pipeline(input_size, output_size):
    return {
        "neural": KerasClassifier(
            nn_classification_model(input_size, output_size, hidden_sizes=[32]),
            #nn_classification_model(input_size, output_size, hidden_sizes=[12]),
            epochs=50, 
            batch_size=32
        ),
        "neural_32_16": KerasClassifier(
            nn_classification_model(input_size, output_size, hidden_sizes=[32, 16]),
            #nn_classification_model(input_size, output_size, hidden_sizes=[12, 12]),
            epochs=50, 
            batch_size=32
        ),
        "neural_32_16_8": KerasClassifier(
            nn_classification_model(input_size, output_size, hidden_sizes=[32, 16, 8]),
            epochs=50, 
            batch_size=32
        ),
        "svm": svm.SVC(shrinking=False, decision_function_shape="ovo", random_state=seed),
        "decision_tree": DecisionTreeClassifier(criterion="entropy", random_state=seed),
        "random_forest": RandomForestClassifier(n_estimators=10, random_state=seed),
        "majority_class": DummyClassifier(strategy="most_frequent", random_state=seed),
        "logistic_regression": LogisticRegression(solver="lbfgs", multi_class="multinomial", random_state=seed)
    }

In [0]:
def save_results(filename, all_results):
    model_names = list(list(all_results.values())[0].keys())

    with open(filename, mode="w") as out_file:
        writer = csv.writer(out_file)
        
        writer.writerow(["Dataset"] + model_names)
        for dataset, results in all_results.items():
            writer.writerow([dataset] + [results[model]["mean_accuracy"] for model in model_names])

In [0]:
datasets = {
    "FA_Binary": dataset_preparation(FA_Binary, 2, attributes),
    "NC_Binary": dataset_preparation(NC_Binary, 2, attributes),
    "Overall_Trio": dataset_preparation(Overall_Trio, 3, attributes)
}

all_results = {}

for dataset_name, dataset in datasets.items():
    print("# Starting train/test procedure for dataset {}".format(dataset_name))
    all_results[dataset_name] = {}
    pipeline = create_pipeline(dataset.num_features, dataset.num_classes)

    for model_name, model in pipeline.items():
        kfold = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=seed)
        scores = cross_val_score(model, dataset.X, dataset.y, cv=kfold)

        all_results[dataset_name][model_name] = {
            "mean_accuracy": scores.mean(),
            "std_accuracy": scores.std()
        }
    
    print("Model\tAccuracy")
    for model, results in all_results[dataset_name].items():
        print(f"{model}\t{results['mean_accuracy']}")
        

# Starting train/test procedure for dataset FA_Binary
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Epoch 1/50
460/460 [==============================] - 1s 3ms/step - loss: 0.7639 - accuracy: 0.5087
Epoch 2/50
460/460 [==============================] - 0s 77us/step - loss: 0.6937 - accuracy: 0.5435
Epoch 3/50
460/460 [==============================] - 0s 72us/step - loss: 0.6487 - accuracy: 0.6022
Epoch 4/50
460/460 [==============================] - 0s 90us/step - loss: 0.6556 - accuracy: 0.6065
Epoch 5/50
460/460 [==============================] - 0s 68us/step - loss: 0.6462 - accuracy: 0.6022
Epoch 6/50
460/460 [==============================] - 0s 73us/step - loss: 0.6237 - accuracy: 0.6370
Epoch 7/50
460/460 [==============================] - 0s 72us/step - loss: 0.6104 - accuracy: 0.6457
Epoch 8/50
460/460 [==============================] - 0s 84us/step - loss: 0.6120 - accuracy: 0.6717
Epoch 9/50
460/460 [==============================] - 0s 

In [0]:
results_file = "experiment_results_all_attributes.csv"

print("# Saving results to file {}".format(results_file))
save_results(results_file, all_results)

# Saving results to file experiment_results_all_attributes.csv


In [0]:
datasets = {
    "FA_Binary": dataset_preparation(FA_Binary, 2, attributes, apply_feature_selection=True),
    "NC_Binary": dataset_preparation(NC_Binary, 2, attributes, apply_feature_selection=True),
    "Overall_Trio": dataset_preparation(Overall_Trio, 3, attributes, apply_feature_selection=True)
}

all_results = {}

for dataset_name, dataset in datasets.items():
    print("# Starting train/test procedure for dataset {}".format(dataset_name))
    all_results[dataset_name] = {}
    pipeline = create_pipeline(dataset.num_features, dataset.num_classes)

    for model_name, model in pipeline.items():
        kfold = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=seed)
        scores = cross_val_score(model, dataset.X, dataset.y, cv=kfold)

        all_results[dataset_name][model_name] = {
            "mean_accuracy": scores.mean(),
            "std_accuracy": scores.std()
        }
    
    print("Model\tAccuracy")
    for model, results in all_results[dataset_name].items():
        print(f"{model}\t{results['mean_accuracy']}")

# Starting train/test procedure for dataset FA_Binary
Epoch 1/50
460/460 [==============================] - 0s 1ms/step - loss: 0.7317 - accuracy: 0.4848
Epoch 2/50
460/460 [==============================] - 0s 95us/step - loss: 0.6928 - accuracy: 0.5217
Epoch 3/50
460/460 [==============================] - 0s 106us/step - loss: 0.6869 - accuracy: 0.5609
Epoch 4/50
460/460 [==============================] - 0s 109us/step - loss: 0.6864 - accuracy: 0.5522
Epoch 5/50
460/460 [==============================] - 0s 127us/step - loss: 0.6853 - accuracy: 0.5674
Epoch 6/50
460/460 [==============================] - 0s 148us/step - loss: 0.6778 - accuracy: 0.5913
Epoch 7/50
460/460 [==============================] - 0s 103us/step - loss: 0.6788 - accuracy: 0.5717
Epoch 8/50
460/460 [==============================] - 0s 104us/step - loss: 0.6774 - accuracy: 0.5848
Epoch 9/50
460/460 [==============================] - 0s 99us/step - loss: 0.6712 - accuracy: 0.6109
Epoch 10/50
460/460 [===========

In [0]:
results_file = "experiment_results_with_feature_selection.csv"

print("# Saving results to file {}".format(results_file))
save_results(results_file, all_results)

# Saving results to file experiment_results_with_feature_selection.csv
